## Import models from Hugging Face hub
This sample shows how to import and register models from [HuggingFace hub](https://huggingface.co/models). 

### How does import work?
The import process runs as a job in your AzureML workspace using components from the `azureml` system registry. The models are downloaded and converted to MLflow packaged format. You can then register the models to your AzureML Workspace or Registry that makes them available for inference or fine tuning. 

### What models are supported for import?
Any model from Hugging Face hub can be downloaded using the `download_model` component. Only the following set of tasks are supported for MLflow conversion:
* fill-mask
* token-classification
* question-answering
* summarization
* text-generation
* text-classification
* translation
* image-classification
* text-to-image

Conversion to MLflow will fail if you attempt to download a model that has a task type other than the above.

### Why convert to MLflow?
MLflow is AzureML's recommended model packaging format. 
* **Inference benefits**: AzureML supports no-code-deployment for models packaged as MLflow that enables a seamless inference experience for the models. Learn more about [MLflow model packaging](https://learn.microsoft.com/en-us/azure/machine-learning/concept-mlflow-models) and [no-code-deployment](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-deploy-mlflow-models-online-endpoints?tabs=sdk). 
* **Fine tuning benefits**: Foundation models imported and converted to MLflow format can be fine tuned using AzureML's fine tuning pipelines. You can use the no-code UI wizards, or the code based job submission with the SDK or CLI/YAML. AzureML's fine tuning pipelines are built using components. This gives you the flexibility to compose your own fine tuning pipelines containing your own jobs for data transformation, post processing and the AzureML fine tuning components. Learn more about pipelines using [sdk](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-create-component-pipeline-python) or [CLI](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-create-component-pipelines-cli).

### What happens if I just download model and register models without converting to MLflow? That's because the task of the model I'm interested in is not among the supported list of tasks.
You can still download and register the model using the outputs of the `download_model` job. You need to [write your own inference code](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-deploy-online-endpoints?tabs=python) in this case. It also means that fine tuning is not yet supported if the task type of the model you are interested in is not in the supported list.

### Outline
* Setup pre-requisites such as compute.
* Pick a model to import.
* Configure the import job.
* Run the fine tuning job.
* Register the fine tuned model. 


**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription - [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace with computer cluster - [Configure workspace](../../configuration.ipynb)
- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](../../../README.md) - check the getting started section


**Motivations** - This notebook explains how to create model importing/publishing pipeline job in workspace using pipeline component registered in a registry

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section, we will connect to the workspace in which the job will be run.

## 1.1 Import the required libraries

In [ ]:
# Import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential, ClientSecretCredential
from azure.ai.ml import UserIdentityConfiguration

from azure.ai.ml import MLClient, Input, Output
from azure.ai.ml.dsl import pipeline
from azure.ai.ml import load_component

## 1.2 Configure credential

We are using `DefaultAzureCredential` to get access to the workspace. 
`DefaultAzureCredential` should be capable of handling most Azure SDK authentication scenarios. 

Reference for more available credentials if it does not work for you: [configure credential example](../../configuration.ipynb), [azure-identity reference doc](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python).

In [ ]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

## 1.3 Get a handle to the workspace and the registry

We use the config file to connect to a workspace. The Azure ML workspace should be configured with a computer cluster. [Check this notebook for configure a workspace](../../configuration.ipynb)

In [ ]:
# Get a handle to workspace from config file
try:
    ml_client_ws = MLClient.from_config(credential=credential)
except:
    ml_client_ws = MLClient(
            credential,
            subscription_id =  "<SUBSCRIPTION_ID>",
            resource_group_name =  "<RESOURCE_GROUP>",
            workspace_name =  "<WORKSPACE_NAME>"
    )

ml_client_registry = MLClient(credential, registry_name="azureml")

# Retrieve an already attached Azure Machine Learning Compute.
cluster_name = "cpu-cluster"
print(ml_client_ws.compute.get(cluster_name))

# 2. Load pipeline component from the registry to create a pipeline


## 2.1 Use registry handle to load the components in workspace

### Components
- import_model - Pipeline component which downloads the model, converts it into mlflow, validates locally and then register it respectively

In [ ]:
import_model = ml_client_registry.components.get(
    name="import_model", version="0.1.0"
)

# 3. Create a pipeline job using the pipeline component

## 3.1 Create pipeline object using necessary parameters


In [10]:
#### Important parameters to pass
- model_id                                
- compute - Compute cluster on which pipeline job will run
- registry_name - This needs to be passed only when you want to register the model in the registry else ignore

#### model_id 

"Browse models on [HuggingFace hub](https://huggingface.co/models) and identify a model to import. Make sure the task type of the model is among the supported tasks as explained in the introduction section. Copy the model id which is available in the URI of the page or can be copied using the copy icon next to the model name and assign it to the variable `MODEL_ID`."


![model-id-screen-shot.png](attachment:model-id-screen-shot.png)

SyntaxError: invalid syntax (3497695939.py, line 3)

In [ ]:
MODEL_ID = "bert-base-cased"
COMPUTE = cluster_name
REGISTRY_NAME = None # Set this value only if you want to register the model in registry

## Check if model already exists 

In [ ]:
try:
    if REGISTRY_NAME:
        models = ml_client_registry.models.list(name = MODEL_ID)
        if models:
            max_version = (max(models_list, key=lambda x: x.version)).version
            model_version = str(int(max_version) + 1)
        print(f"Model already exists in {REGISTRY_NAME} with name {MODEL_ID} and version {model_version}")
    else:
        model = ml_client_ws.models.get(name = MODEL_ID, label = "latest")
        print(f"Model already exists in your workspace with name {MODEL_ID} and version {model.version}")
except:
    print("Model does not exist ! Import the model using pipeline component")

## Import the model

In [ ]:
pipeline_object = import_model(
                    model_id = MODEL_ID,
                    compute = COMPUTE,
                    task_name ="cdcdcd"
                    # registry_name = "azureml-preview-test1", pass only if you want to register model in registry
                    )

## 3.2 Setting pipeline computes,identity 

In [ ]:
# Settings of compute and Identity
pipeline_object.settings.force_rerun = True
pipeline_object.settings.default_compute  = cluster_name
pipeline_object.identity = UserIdentityConfiguration()

# 4. Submit the import job


In [ ]:
# submit the pipeline job
pipeline_job = ml_client_ws.jobs.create_or_update(pipeline_object, experiment_name=f"bert-base-cased import")
# wait for the pipeline job to complete
ml_client_ws.jobs.stream(pipeline_job.name)